In [1]:
import requests #  collect the page from the web
from bs4 import BeautifulSoup # parse the page
from fake_useragent import UserAgent
import re # regular expression

In [2]:
url = "https://www.flipkart.com/kutchina-30-litre-zephire-30-l-oven-toaster-grill-otg/p/itm3d3c6ab9d4d21?pid=OTNG588ZAWHPJYDN&lid=LSTOTNG588ZAWHPJYDNHB6R6C&marketplace=FLIPKART&store=j9e&srno=b_1_1&otracker=hp_omu_Deals%2Bof%2Bthe%2BDay_3_4.dealCard.OMU_4ZMLVX21QJTE_3&otracker1=hp_omu_SECTIONED_manualRanking_neo%2Fmerchandising_Deals%2Bof%2Bthe%2BDay_NA_dealCard_cc_3_NA_view-all_3&fm=neo%2Fmerchandising&iid=en_V%2F0BkRFD1boi3cNwKkdgc0piprvwej05Yr6e5hVxQqkt%2BngFnJt8PPGlzYydDvbbUI9KK8j0uR3%2BJbDGNhRlAA%3D%3D&ppt=browse&ppn=browse&ssid=hcsdbg8akw0000001654579958364"
ua = UserAgent()
headers = {'User-Agent': ua.random}
page = requests.get(
    url, 
    headers=headers,
    cookies={
        "session-id":"", "session-id-time":"", "session-token":""
    })
print(page)
if page.status_code == 404:
    print("Page not found")
elif page.status_code == 503:
    print("Service unavailable")
elif page.status_code == 200:
    print("Page found")
    soup = BeautifulSoup(page.content, 'html.parser')
    

<Response [200]>
Page found


In [3]:
def validate_url(url):
    regex = re.compile(r'^(?:http)s?://',re.IGNORECASE)
    return re.match(regex, url) is not None

def clean_url(url):
    if '?' in url:
        url = url.split('?')[0]
    return url

In [4]:
url = "https://www.amazon.in/s?k=headphones&crid=2VVV3IUUC62ND&sprefix=headphones%2Caps%2C220&ref=nb_sb_noss_1"
c_url = clean_url(url) 
print(c_url)
validate_url(url)

https://www.amazon.in/s


True

In [11]:
def get_page(url,headers=None,cookies=None,clean=True):
    if clean:
        url = clean_url(url)
    if not validate_url(url):
        print(f"Invalid url : {url}")
        return None
    ua = UserAgent()
    if headers is None:
        headers = {'User-Agent': ua.random}
    if cookies is None:
        cookies = {
            "session-id":"", "session-id-time":"", "session-token":""
        }
    page = requests.get(url,headers=headers,cookies=cookies)
    if page.status_code == 404:
        print("Page not found")
        return None
    elif page.status_code == 503:
        print("Service unavailable")
        return None
    elif page.status_code == 200:
        soup = BeautifulSoup(page.content, 'html.parser')
        return soup

In [8]:
soup = get_page(url)

In [ ]:
soup

In [12]:
soup = get_page(url="https://www.amazon.in/s?k=headphones&crid=2VVV3IUUC62ND&sprefix=headphones%2Caps%2C220&ref=nb_sb_noss_1",clean=False)

# soup data extraction

In [13]:
type(soup)

bs4.BeautifulSoup

In [14]:

if isinstance(soup, BeautifulSoup):
    print('item can be used')
else:
    print("error")

item can be used


In [10]:
soup.find('span',attrs={'id':'productTitle'}).text

'        Logitech B100 Wired USB Mouse, 3 yr Warranty, 800 DPI Optical Tracking, Ambidextrous PC/Mac/Laptop - Black       '

In [11]:
soup.find('span',attrs={'class':'a-price-whole'}).text

'245.'

In [12]:
soup.find('a',attrs={'id':'bylineInfo'}).attrs.get('href')

'/stores/Logitech/page/EB4825FF-261C-4334-A53E-A746981F7C24?ref_=ast_bln'

In [26]:
def extract_one(soup, **selectors):
    if not isinstance(soup,BeautifulSoup):
        print("Not a BeautifulSoup object")
        return None
    data = {}
    for key,info in selectors.items():
        tag = info.get('t','div')
        attrs = info.get('a',None)
        out = info.get('o','text')
        if out == 'text':
            data[key] = soup.find(tag,attrs=attrs).text
        elif out == 'href':
            data[key] = soup.find(tag,attrs=attrs).attrs.get('href')
        elif out == 'src':
            data[key] = soup.find(tag,attrs=attrs).attrs.get('src')
    return data
    

In [27]:
titleDict = {'t':'span','a':{'id':'productTitle'}}
priceDict = {'t':'span','a':{'class':'a-price-whole'}}
linkDict = {'t':'a','a':{'id':'bylineInfo'},'o':'href'}
totalRating = dict(t='span',a={'id':'acrCustomerReviewText'})
print(extract_one(soup,title=titleDict,price=priceDict,link=linkDict,total_rating=totalRating))

{'title': '        Logitech B100 Wired USB Mouse, 3 yr Warranty, 800 DPI Optical Tracking, Ambidextrous PC/Mac/Laptop - Black       ', 'price': '245.', 'link': '/stores/Logitech/page/EB4825FF-261C-4334-A53E-A746981F7C24?ref_=ast_bln', 'total_rating': '26,377 ratings'}


extracting a list

In [16]:
# target area that contains the displayed data
target = soup.find('div',attrs={'class':'s-main-slot s-result-list s-search-results sg-row'})
if target is None:
    print("No data found")


In [20]:
items = target.find_all('div',attrs={'class':'s-result-item'})
item_count = len(items)
if item_count == 0:
    print("No data found")
else:
    print(f"{item_count} items found")

31 items found


In [40]:
data_list = []
for item in items:
    try:
        title = item.find('h2',attrs={'class':'a-size-mini a-spacing-none a-color-base s-line-clamp-2'}).text
    except:
        title = None
    try:
        price = item.find('span',attrs={'class':'a-price-whole'}).text
    except:
        price = None
    try:
        rating = item.find('span',attrs={'class':'a-icon-alt'}).text
    except:
        rating = None
    if title is not None and price is not None and rating is not None:
        data_list.append({
            'title':title,
            'price':price,
            'rating':rating
        })

In [44]:
import pandas as pd

In [45]:
pd.DataFrame(data_list)

,title,price,rating
0,"Infinity (JBL) Glide 510, 72 Hrs Playtime with...","1,499",4.2 out of 5 stars
1,Lenovo 110 Wired On Ear Headphones with Mic (W...,"1,699",4.2 out of 5 stars
2,boAt Rockerz 550 Bluetooth Wireless Over Ear H...,"1,799",4.1 out of 5 stars
3,boAt Bassheads 100 Wired in Ear Earphones with...,379,4.1 out of 5 stars
4,boAt Rockerz 450 Bluetooth Wireless On Ear Hea...,"1,499",4.1 out of 5 stars
5,Logitech H111 Wired Over Ear Headphones With M...,798,4.0 out of 5 stars
6,boAt Rockerz 450 Bluetooth Wireless On Ear Hea...,"1,399",4.1 out of 5 stars
7,boAt Bassheads 900 Wired On Ear Headphones wit...,799,4.2 out of 5 stars
8,boAt Rockerz 510 Bluetooth Wireless Over Ear H...,"1,399",4.2 out of 5 stars
9,pTron Pride Lite HBE (High Bass Earphones) in-...,199,3.6 out of 5 stars
